In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *


In [ ]:
#| code-summary: import all the packages needed for the project
#| output: hide

from ids_finder.utils import *
from ids_finder.core import *
from fastcore.utils import *
from fastcore.test import *

import polars as pl
try:
    import modin.pandas as pd
    import modin.pandas as mpd
except ImportError:
    import pandas as pd

import pandas
import numpy as np
from humanize import naturalsize
import xarray as xr


from datetime import timedelta
from loguru import logger
import speasy as spz


## Dataset Overview

In [ ]:
artemis_probes = ["b", "c"]
probe = artemis_probes[0]

jno_start_date = "2011-08-25"
jno_end_date = "2016-06-30" 

trange = [jno_start_date, jno_end_date]
test_trange = ["2011-08-25", "2011-09-25"]

### Download all the files

In [ ]:
sat = 'thb'
coord = 'gse'
datatype = 'fgs'

sat_fgm_product = f'cda/{sat.upper()}_L2_FGM/{sat}_fgs_gse'
sat_pos_sse_product = f'cda/{sat.upper()}_L1_STATE/{sat}_pos_sse'
sat_pos_gse_product = f'cda/{sat.upper()}_L1_STATE/{sat}_pos_gse'

products = [
    sat_fgm_product,
    sat_pos_sse_product,
    sat_pos_gse_product
]

In [ ]:
@threaded
def download_data(products, trange):
    logger.info("Downloading data")
    spz.get_data(products, trange, progress=True, disable_proxy=True)
    logger.info("Data downloaded")
    # spz.get_data(products, jno_start_date, jno_end_date)   

Download data in a background thread

In [3]:
%%markdown
#| eval: false
download_data(products, trange)

#| eval: false
download_data(products, trange)


### Convert data to `parquet` for faster processing

In [ ]:
def thm_rename_col(col: str):
    if "," in col:
        col = col.split(",")[0]
    return col.split()[0].upper()

def spz2parquet(data, force=False):
    output = f"../data/{data.name}.parquet"
    if Path(output).exists() and not force:
        logger.info("Data already converted to parquet")
    else: 
        df = pandas.DataFrame(
            data.values, index=pandas.Series(data.time, name="time"), columns=data.columns
        )
        
        df.to_parquet(output)
        logger.info("Data converted to parquet successfully")

In [4]:
%%markdown

dataset = spz.get_data(products, trange)

for data in dataset:
    spz2parquet(data, force=False)


dataset = spz.get_data(products, trange)

for data in dataset:
    spz2parquet(data, force=False)


## Processing the whole data

In [ ]:
def get_thm_state(sat):
    sat_pos_sse_files = f"../data/{sat}_pos_sse.parquet"
    sat_pos_sse = pl.scan_parquet(sat_pos_sse_files).set_sorted("time")
    sat_pos_gse_files = f"../data/{sat}_pos_gse.parquet"
    sat_pos_gse = pl.scan_parquet(sat_pos_gse_files).set_sorted("time")
    sat_state = sat_pos_sse.join(sat_pos_gse, on="time", how="inner")
    return sat_state

In [ ]:
#| eval: false

sat = "thb"
coord = "gse"
datatype = "fgs"
tau = timedelta(seconds=60)
data_resolution = timedelta(seconds=4)

files = f"../data/{sat}_{datatype}_{coord}.parquet"
output = f"../data/{sat}_candidates_tau_{tau.seconds}.parquet"

rename_mapping = {
    "Bx FGS-D": "BX",
    "By FGS-D": "BY",
    "Bz FGS-D": "BZ",
}

data = pl.scan_parquet(files).rename(rename_mapping).unique("time").sort("time").collect()
sat_fgm = df2ts(
    data, ["BX", "BY", "BZ"], attrs={"coordinate_system": coord, "units": "nT"}
)
sat_state = get_thm_state(sat)

indices = compute_indices(data, tau)

# # filter condition
sparse_num = tau / data_resolution // 3
filter_condition = get_ID_filter_condition(sparse_num = sparse_num)

candidates_pl = indices.filter(filter_condition).with_columns(pl_format_time(tau))
candidates = convert_to_dataframe(candidates_pl)
del indices

In [ ]:
#| eval: false
def get_memory_usage(data):
    datatype = type(data)
    match datatype:
        case pl.DataFrame:
            size = data.estimated_size()
        case pd.DataFrame:
            size = data.memory_usage().sum()
        case xr.DataArray:
            size = data.nbytes

    logger.info(f"{naturalsize(size)} ({datatype.__name__})")
    return size


get_memory_usage(indices)
get_memory_usage(candidates)
get_memory_usage(sat_fgm)

NameError: name 'sat_fgm' is not defined

In [ ]:
#| eval: false

ids = process_candidates(candidates, sat_fgm, sat_state, data_resolution)


(raylet) [2023-09-24 00:55:26,352 E 70415 3620643] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-09-24_00-49-06_800983_67101 is over 95% full, available space: 24609177600; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2023-09-24 00:55:36,360 E 70415 3620643] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-09-24_00-49-06_800983_67101 is over 95% full, available space: 23729713152; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2023-09-24 00:55:46,375 E 70415 3620643] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-09-24_00-49-06_800983_67101 is over 95% full, available space: 23729610752; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2023-09-24 00:55:56,451 E 70415 3620643] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-09-24_00-49-06_800983_67101 is over 95% full, available space: 23728783360; capacity: 494384795648. Object 

PipelineApplicationError: Exception raised in stage [ 1] PdPipelineStage: Generating a column with a function calculating duration parameters if needed.

(raylet) [2023-09-24 00:56:06,481 E 70415 3620643] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-09-24_00-49-06_800983_67101 is over 95% full, available space: 23729135616; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2023-09-24 00:56:16,549 E 70415 3620643] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-09-24_00-49-06_800983_67101 is over 95% full, available space: 23730778112; capacity: 494384795648. Object creation will fail if spilling is required.


## Obsolete codes

In [ ]:
#| eval: false
import pycdfpp
import pyspedas

In [ ]:
#| eval: false

def convert_thm_state_to_parquet(
    probe: str, trange
):
    file_name = f"./data/th{probe}_state.parquet"
    if os.path.exists(file_name):
        return file_name

    start = trange.start.to_string()
    end = trange.end.to_string()

    files = pyspedas.themis.state(
        probe=probe,
        trange=[start, end],
        downloadonly=True,
        no_update=True,
    )

    thm_pos_sse_Xs = []
    thm_pos_gse_Xs = []
    thm_state_times = []
    for file in files:
        thm_state = pycdfpp.load(file)
        epoch_dt64 = thm_state[
            f"time"
        ].values  #  CATDESC: "thm_state_time, UTC, in seconds since 01-Jan-1970 00:00:00"
        thm_pos_sse_Xs.append(thm_state[f"th{probe}_pos_sse"].values[:, 0])
        thm_pos_gse_Xs.append(thm_state[f"th{probe}_pos_gse"].values[:, 0])
        thm_state_times.append(epoch_dt64)

    thm_pos_sse_X = np.concatenate(thm_pos_sse_Xs)
    thm_pos_gse_X = np.concatenate(thm_pos_gse_Xs)
    thm_state_time = np.concatenate(thm_state_times)

    pl.DataFrame(
        {
            "thm_state_time": thm_state_time,
            "thm_pos_gse_X": thm_pos_gse_X,
            "thm_pos_sse_X": thm_pos_sse_X,
        }
    ).with_columns(
        pl.from_epoch(pl.col("thm_state_time"), time_unit="s")
    ).write_parquet(
        file_name
    )

    return file_name


def convert_thm_fgm_to_parquet(probe, trange):
    file_name = f"./data/th{probe}_fgm.parquet"
    if os.path.exists(file_name):
        return file_name

    start = trange.start.to_string()
    end = trange.end.to_string()
    
    files = pyspedas.themis.fgm(
        probe=probe,
        trange=[start, end],
        downloadonly=True,
        no_update=True,
    )

    thm_fgl_gses = []
    thm_fgl_btotals = []
    thm_fgl_times = []

    for file in files:
        cdf = pycdfpp.load(file)
        thm_fgl_gses.append(cdf[f"th{probe}_fgl_gse"].values)
        thm_fgl_btotals.append(cdf[f"th{probe}_fgl_btotal"].values)
        thm_fgl_times.append(cdf[f"th{probe}_fgl_time"].values)

    thm_fgl_gse = np.concatenate(thm_fgl_gses)
    thm_fgl_btotal = np.concatenate(thm_fgl_btotals)
    thm_fgl_time = np.concatenate(thm_fgl_times)

    pl.DataFrame(
        {
            "time": thm_fgl_time,
            "BX": thm_fgl_gse[:,0],
            "BY": thm_fgl_gse[:,1],
            "BZ": thm_fgl_gse[:,2],
            "B": thm_fgl_btotal,
        }
    ).with_columns(
        pl.from_epoch(pl.col("thm_fgl_time"), time_unit="s"),
    ).write_parquet(   
        file_name
    )
    
    return file_name

In [ ]:
#| eval: false
convert_thm_state_to_parquet(probe, trange)
convert_thm_fgm_to_parquet(probe, trange)